In [1]:
%pwd

'e:\\github_clone\\Text-Summarization-English\\research'

In [2]:
import os
os.chdir('../')

In [11]:
%pwd

'e:\\github_clone\\Text-Summarization-English'

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationconfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [13]:
from textSummerizerEnglish.constants import *
from textSummerizerEnglish.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationconfig:
        config = self.config.data_transformation

        data_transformation_config = DataTransformationconfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )
        return data_transformation_config

In [15]:
from textSummerizerEnglish.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, Dataset
import pandas as pd

In [18]:
class DataTransformation:
    def __init__(self, config:DataTransformationconfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['ctext'] , max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['text'], max_length = 128, truncation = True )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        train_df = pd.read_csv(os.path.join(self.config.data_path, "train", "train.csv"),encoding="latin-1")
        train_df = train_df.dropna()
        train_data = Dataset.from_pandas(train_df)

        test_df = pd.read_csv(os.path.join(self.config.data_path, "test", "test.csv"),encoding="latin-1")
        test_df = test_df.dropna()
        test_data = Dataset.from_pandas(test_df)

        val_df = pd.read_csv(os.path.join(self.config.data_path, "val", "val.csv"),encoding="latin-1")
        val_df = val_df.dropna()
        val_data = Dataset.from_pandas(val_df)


        train_pt = train_data.map(self.convert_examples_to_features, batched = True)
        test_pt = test_data.map(self.convert_examples_to_features, batched = True)
        val_pt = val_data.map(self.convert_examples_to_features, batched = True)

        train_pt.save_to_disk(os.path.join(self.config.root_dir, "new_summary", "train"))
        test_pt.save_to_disk(os.path.join(self.config.root_dir, "new_summary", "test"))
        val_pt.save_to_disk(os.path.join(self.config.root_dir, "new_summary", "val"))

In [19]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2023-12-04 17:10:27,825: INFO: yaml file : config\config.yaml loaded successfully]
[2023-12-04 17:10:27,827: INFO: yaml file : params.yaml loaded successfully]
[2023-12-04 17:10:27,828: INFO: Created directory at artifacts]


Map:   0%|          | 0/2865 [00:00<?, ? examples/s]c:\Users\Ernest\.conda\envs\textS\lib\site-packages\transformers\tokenization_utils_base.py:3864: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 763/763 [00:00<00:00, 127347.95 examples/s]
